In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/workspaces/stok_opt-mimizasyon_mlops_project'

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("artifacts/data_ingestion/satslar.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 22 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   InventoryId              1048575 non-null  object 
 1   Store                    1048575 non-null  int64  
 2   Brand                    1048575 non-null  int64  
 3   Description_sales        1048575 non-null  object 
 4   Size_sales               1048575 non-null  object 
 5   SalesQuantity            1048575 non-null  int64  
 6   SalesDollars             1048575 non-null  float64
 7   SalesPrice               1048575 non-null  float64
 8   SalesDate                1048575 non-null  object 
 9   Volume_sales             1048575 non-null  int64  
 10  Classification_sales     1048575 non-null  int64  
 11  ExciseTax                1048575 non-null  float64
 12  VendorNo                 1048575 non-null  int64  
 13  VendorName_sales         1048575 non-null 

In [6]:
data.columns

Index(['InventoryId', 'Store', 'Brand', 'Description_sales', 'Size_sales',
       'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate',
       'Volume_sales', 'Classification_sales', 'ExciseTax', 'VendorNo',
       'VendorName_sales', 'Description_purchase', 'Price', 'Size_purchase',
       'Volume_purchase', 'Classification_purchase', 'PurchasePrice',
       'VendorNumber', 'VendorName_purchase'],
      dtype='object')

# ENTİTİY

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

# CONFİG

In [8]:
from src.stok_optimizasyonu_ml_project.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.stok_optimizasyonu_ml_project.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath= SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )

        return data_validation_config

# COMPONENTS

In [10]:
import os
from src.stok_optimizasyonu_ml_project import logger

In [11]:
import pandas as pd
from scipy.stats import zscore

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self) -> bool:
        try:
            validation_status = True  # Başlangıçta doğrulama başarılı (True)
            
            # Veriyi okuyoruz
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)  # Kolon isimlerini alıyoruz
            all_schema = self.config.all_schema.keys()  # Şemadaki kolon isimleri
            
            # Kolonları kontrol et
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False  # Kolon eksikse False dönecek
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}, Error: Missing Column - {col}")
                    return validation_status  # Eğer bir kolon eksikse işlemi sonlandırıyoruz

            # Kolon sırasını kontrol et (Eğer gereksinim varsa)
            expected_columns = list(self.config.all_schema.keys())
            if all_cols != expected_columns:
                validation_status = False
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(f"Validation status: {validation_status}, Error: Column Order mismatch.")

            # Eksik veri kontrolü
            missing_data = data.isnull().sum()  # Her kolondaki eksik veri sayısını alıyoruz
            if missing_data.any():  # Eğer eksik veri varsa
                validation_status = False
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(f"Validation status: {validation_status}, Missing Data: {missing_data}")
            
            # Aykırı Değer Analizi (Outlier Analysis)
            outlier_data = self.outlier_analysis(data, threshold=1.5)  # Aykırı değer analizi
            if outlier_data:
                validation_status = False
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(f"Validation status: {validation_status}, Outliers found: {outlier_data}")

            # Z-Score Aykırı Değer Analizi
            outliers_dict = self.zscore_analysis_multi(data, columns=all_cols, threshold=3)  # Z-score ile aykırı değer analizi
            if outliers_dict:
                validation_status = False
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(f"Validation status: {validation_status}, Z-score outliers: {outliers_dict}")

            # Validation durumu yazıyoruz
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")
            
            return validation_status  # Sonuç döndürülüyor
        except Exception as e:
            raise e  # Hata durumunda istisna fırlatılır

    # Aykırı değer tespiti (IQR - Interquartile Range) ile
    def outlier_analysis(self, df, threshold=1.5):
        outlier_data = {}
        numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()  # Sayısal kolonları seçiyoruz

        for column in numerical_columns:
            Q1 = df[column].quantile(0.25)  # 1st quartile (Q1)
            Q3 = df[column].quantile(0.75)  # 3rd quartile (Q3)
            IQR = Q3 - Q1  # Interquartile Range (IQR)

            lower_limit = Q1 - threshold * IQR  # Alt sınır
            upper_limit = Q3 + threshold * IQR  # Üst sınır

            # Aykırı değerleri tespit ediyoruz
            outlier_mask = (df[column] < lower_limit) | (df[column] > upper_limit)
            outlier_data[column] = {
                "lower_limit": lower_limit,
                "upper_limit": upper_limit,
                "outlier_count": outlier_mask.sum(),  # Aykırı değerlerin sayısı
                "percentage": round((outlier_mask.sum() / len(df)) * 100, 2)  # Yüzde oranı
            }

        return outlier_data  # Aykırı değerleri döndürüyoruz

    # Z-score ile aykırı değer analizi (Sayısal kolonlar için)
    def zscore_analysis_multi(self, df, columns, threshold=3):
        outliers_dict = {}

        for column in columns:
            df[f'{column}_zscore'] = zscore(df[column])  # Z-score hesaplama

            # Z-score değeri belirli bir eşik değerini aşarsa aykırı değer olarak işaretliyoruz
            outlier_mask = (df[f'{column}_zscore'] > threshold) | (df[f'{column}_zscore'] < -threshold)

            outliers_dict[column] = df[outlier_mask]  # Aykırı değerleri kaydediyoruz

        return outliers_dict  # Aykırı değerlerin bulunduğu dictionary döndürülüyor

# PİPELİNE

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    
    # Veri doğrulamasını yapıyoruz
    validation_status = data_validation.validate_all_columns()
    
    if not validation_status:
        # Validation başarısızsa, log kaydı ekleyebiliriz
        with open(data_validation.config.STATUS_FILE, 'a') as f:  # Burada 'self' yerine 'data_validation.config' kullanıyoruz
            f.write(f"Data validation failed at {pd.to_datetime('now')}\n")
    
except Exception as e:
    # Hata mesajını dosyaya yazabiliriz
    with open(data_validation.config.STATUS_FILE, 'a') as f:  # Burada da aynı şekilde 'self' yerine 'data_validation.config' kullanıyoruz
        f.write(f"Error occurred: {str(e)}\n")
    raise  # Hata tekrar raise ediliyor, böylece program daha üst seviyede yakalanabilir.